In [12]:
import os
import cv2
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

# Load the model
model = YOLO("./models/yolov8x.pt")
names = model.names

results = None

# Load the image
image_path = ["img1.jpg", "img2.jpg"]
for i in image_path:
    im0 = cv2.imread(i)
    # Check if the image was successfully loaded
    assert im0 is not None, "Error reading image file"
    
    # Create a base directory for cropped images
    crop_base_dir = "cropped_objects"
    if not os.path.exists(crop_base_dir):
        os.mkdir(crop_base_dir)
    
    # Predict objects in the image
    results = model.predict(im0, show=False, device=0)
    # Extract bounding boxes and class IDs from the results
    boxes = results[0].boxes.xyxy.cpu().tolist()
    clss = results[0].boxes.cls.cpu().tolist()
    
    # Initialize a counter for cropped images
    idx = 0
    # Dictionary to keep count of objects for each class
    class_counts = {}
    
    # Check if there are any detected objects
    if boxes is not None:
        for box, cls in zip(boxes, clss):
            idx += 1
    
            # Crop the detected object from the image
            crop_obj = im0[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
    
            # Get the class name for the detected object
            class_name = names[int(cls)]
            # Create a directory for the class if it doesn't exist
            class_dir = os.path.join(crop_base_dir, names[int(cls)])
            if not os.path.exists(class_dir):
                os.mkdir(class_dir)
    
            # Save the cropped object in the class directory
            cv2.imwrite(os.path.join(class_dir, f"{i.split('.')[0]}_{idx}.png"), crop_obj)

            # Update the class count
            if class_name in class_counts:
                class_counts[class_name] += 1
            else:
                class_counts[class_name] = 1

    # Create an annotator for visualizing the results
    annotator = Annotator(im0, line_width=2, example=names)
    # Check if there are any detected objects
    if boxes is not None:
        for box, cls in zip(boxes, clss):
            # Annotate the bounding box on the image
            annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])
    
    # Display the annotated image
    cv2.imshow("ultralytics", im0)
    
    # Save the annotated image
    cv2.imwrite(f"{i.split('.')[0]}_object_cropping_output.jpg", im0)

    # Wait for a key press and then close all OpenCV windows
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Print the counts of objects per class
    for class_name, count in class_counts.items():
        print(f"Class '{class_name}' has {count} objects.")


0: 448x640 13 persons, 1 bicycle, 3 cars, 2 traffic lights, 3 backpacks, 7 handbags, 104.5ms
Speed: 3.0ms preprocess, 104.5ms inference, 3.0ms postprocess per image at shape (1, 3, 448, 640)
Class 'person' has 13 objects.
Class 'car' has 3 objects.
Class 'handbag' has 7 objects.
Class 'backpack' has 3 objects.
Class 'traffic light' has 2 objects.
Class 'bicycle' has 1 objects.

0: 480x640 5 persons, 1 bottle, 1 cup, 3 chairs, 2 potted plants, 1 vase, 346.1ms
Speed: 3.0ms preprocess, 346.1ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)
Class 'person' has 5 objects.
Class 'vase' has 1 objects.
Class 'potted plant' has 2 objects.
Class 'chair' has 3 objects.
Class 'bottle' has 1 objects.
Class 'cup' has 1 objects.
